# word2vecの挙動を確認する

In [1]:
from gensim.models import KeyedVectors

In [2]:
model = KeyedVectors.load_word2vec_format('model.vec', binary=True)

print('model loaded')

model loaded


In [3]:
from gensim.models.word2vec import Word2Vec

In [4]:
model['MYOPE_QA_ID:0']

KeyError: "word 'MYOPE_QA_ID:0' not in vocabulary"

ボキャブラリがない場合はエラーになる

In [5]:
from gensim.similarities.docsim import WmdSimilarity
import numpy as np

corpus = np.array([
    ['こんにちは', '交通', '費', '申請'],
    ['海外', '日当' 'MYOPE_QA_ID:888'],
    ['国内', '出張', '担当', '者', '教え', '下さい'],
    ['海外', '出張', '申請', 'ルート', '承認', '者', '変更', 'MYOPE_QA_ID:999']
])

In [6]:
wmd = WmdSimilarity(corpus, model, num_best=10)

In [7]:
wmd['MYOPE_QA_ID:999']

[(3, 0.42747335809607512),
 (0, 0.42612267237868551),
 (1, 0.41935011413942114),
 (2, 0.41906706063564897)]

In [8]:
wmd[['海外', '日当', 'MYOPE_QA_ID:0']]

[(1, 0.60361194695176656),
 (3, 0.47628312326919181),
 (2, 0.44740058255077148),
 (0, 0.44600532859279046)]

In [9]:
wmd[['海外', '日当']]

[(1, 0.60361194695176656),
 (3, 0.47628312326919181),
 (2, 0.44740058255077148),
 (0, 0.44600532859279046)]

In [10]:
wmd[['海外', '日当', 'MYOPE_QA_ID:888']]

[(1, 0.60361194695176656),
 (3, 0.47628312326919181),
 (2, 0.44740058255077148),
 (0, 0.44600532859279046)]

MYOPE_QA_IDを変えても同じスコアが出た

wmd自体には他にメソッドがないため確認できなかったが、MYOPE_QA_IDを無視されてそうな挙動

In [16]:
w2v = Word2Vec()
print(w2v.wmdistance(['海外'], ['日当']))
w2v.wv = model
print(w2v.wmdistance(['海外'], ['日当']))
print(w2v.wmdistance(['MYOPE_QA_ID:888'], ['日当']))
print(w2v.wmdistance(['MYOPE_QA_ID:888'], ['MYOPE_QA_ID:999']))
sentences = [['海外', '出張', '申請', 'ルート', '承認', '者', '変更', 'MYOPE_QA_ID:999'], ['海外', '日当' 'MYOPE_QA_ID:888']]
w2v.scan_vocab(sentences)
w2v.build_vocab(sentences, update=True)
print(w2v.wmdistance(['ほげほげ', 'MYOPE_QA_ID:888'], ['日当']))

inf
1.3133870363235474
inf
inf


RuntimeError: cannot sort vocabulary after model weights already initialized.

https://github.com/RaRe-Technologies/gensim/issues/1162

> 単語がない場合（おそらく、新しい素材が以前のトレーニングでは非常に小さいか、または非常に冗長であるため）、後続のトレーニングがモデルにとって実質的なメリットではない場合もあります。 （このインクリメンタル・ボキャブラリー拡張オプションは実験的な機能と見なされるのが最善であり、使用ごとに慎重に評価する必要があります。実際の「オンライン」トレーニング・オプションではありません。小さな/冗長な新しいバッチを使用してupdate-vocabを簡単に呼び出すプロセスには注意が必要です。

vocablaryを追加する方法、このエラーの解決法がわからない